In [57]:
import artm
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

In [48]:
f = open("./vw.wiki-enru.txt", "r")

In [49]:
lines = f.readlines()

In [55]:
print(lines[8])

Нэрима_(станция)|Nerima_Station |@russian островной мимо линия:4 использоваться боковой номер tokyo железнодорожный открывать планировка тосима станция:7 путь:2 близлежащий специальный railway:2 обозначать metropolitan токио bureau оэдо:4 transportation платформа:3 тип:2 открываться метрополитен toei юракут турникет район |@english serving:2 numbering consists japan ground outer daily:2 ikebukuro:2 platform board increased stop ticket:2 layout level busiest either inter escalators elevators platforms:4 connect pass operated shown opened:2 per waiting previous subway:2 tracks:3 entrances toshima stations figures passing introduced via private toei:8 operator statistics additional contains station:17 rebuilt passengers becoming commenced:2 train terminus room tokyo below island:2 toyoko tokyu passenger:2 information:2 elevated:2 adjacent underground:2 gate track trains:2 running:2 services railway:5 network fiscal:4 lines:2 average:3 km seibu:11 si allow



In [ ]:
batch_vectorizer = artm.BatchVectorizer(data_path='./vw.wiki-enru.txt',
                                        data_format='vowpal_wabbit',
                                        target_folder='my_collection_batches')

In [ ]:
dictionary = artm.Dictionary()
dictionary.gather(data_path='my_collection_batches')

In [ ]:
dictionary.save_text(dictionary_path='my_collection_batches/my_dictionary.txt')
#dictionary.save(dictionary_path='/my_collection_batches/my_dictionary')

In [ ]:
#dictionary.load_text(dictionary_path='my_collection_batches/my_dictionary.txt')

In [ ]:
model = artm.ARTM(num_topics=20, dictionary=dictionary)

In [ ]:
model.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=10)

In [13]:
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            #if word in stopwords.words('russian'):
            #    continue
            if len(word) >= 2:
                tokens.append(word.lower())
    return tokens

In [14]:
fp = open("./paper.txt", "r")
paper = fp.readlines()

In [15]:
text = ""
for x in paper:
    text = text + x

In [16]:
print(text)

Квас (укр. , белор. , макед. , сербохорв. квас, сербохорв. диал. аловина, словацк. kysel', kyselica, parovec, польск. kwas, kisiel, żur), лит. gira), латыш. kvass) — традиционный славянский кислый напиток, который готовят на основе брожения из муки и солода (ржаного, ячменного) или из сухого ржаного хлеба, иногда с добавлением пахучих трав, мёда, вощины; также готовится из свёклы, фруктов, ягод. Раньше он нередко служил основой холодных похлёбок[1].
В период между IX и XIV веками славяне изготавливали алкогольный квас творёный (свареный), и слово «квасник» на языке того времени употребляли в значении «пьяница»[2]. Лужиц. kwas «праздник, свадебный пир»[3] и ст.-словацк. kvas «пир, угощение, гуляние»[1] свидетельствуют о том, что раньше квас был основным хмельным напитком на праздниках и свадьбах[1].



In [17]:
tok_text = tokenize_text(text.decode('utf-8'))

In [19]:
for word in tok_text:
    print(word)

квас
укр
белор
макед
сербохорв
квас
сербохорв
диал
аловина
словацк
kysel
kyselica
parovec
польск
kwas
kisiel
żur
лит
gira
латыш
kvass
традиционный
славянский
кислый
напиток
который
готовят
на
основе
брожения
из
муки
солода
ржаного
ячменного
или
из
сухого
ржаного
хлеба
иногда
добавлением
пахучих
трав
мёда
вощины
также
готовится
из
свёклы
фруктов
ягод
раньше
он
нередко
служил
основой
холодных
похлёбок
период
между
ix
xiv
веками
славяне
изготавливали
алкогольный
квас
творёный
свареный
слово
«квасник»
на
языке
того
времени
употребляли
значении
«пьяница»
лужиц
kwas
«праздник
свадебный
пир»
ст.-словацк
kvas
«пир
угощение
гуляние»
свидетельствуют
том
что
раньше
квас
был
основным
хмельным
напитком
на
праздниках
свадьбах


In [33]:
count_vectorizer = CountVectorizer(
    analyzer="word", tokenizer=nltk.word_tokenize,
    preprocessor=None, max_features=3000) 
train_data_features = count_vectorizer.fit_transform(paper)

In [61]:
tt = train_data_features.toarray()
vocab = count_vectorizer.get_feature_names()

In [66]:
# Sum up the counts of each vocabulary word
dist = np.sum(tt, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
out_txt = ""
for tag, count in zip(vocab, dist):
    if count == 1:
        out_txt = out_txt + tag + " "
    else:
        out_txt = out_txt + tag + ":" + str(count) + " "

In [67]:
print(out_txt)

' (:3 ):5 ,:24 .:16 1:3 2 3 ; [:5 ]:5 gira ix kisiel kvas kvass kwas:2 kysel kyselica parovec xiv «квасник» «пир «праздник «пьяница» żur алкогольный аловина белор брожения был в:2 веками вощины времени готовится готовят гуляние» диал добавлением значении и:5 из:3 изготавливали или иногда квас:4 кислый который латыш лит лужиц макед между муки мёда на:3 напитком напиток нередко о он основе основным основой пахучих период пир» польск похлёбок праздниках раньше:2 ржаного:2 с свадебный свадьбах свареный свидетельствуют свёклы сербохорв:2 славяне славянский словацк слово служил солода ст.-словацк сухого также творёный того том трав традиционный угощение укр употребляли фруктов хлеба хмельным холодных что ягод языке ячменного — 


In [36]:
for x in count_vectorizer.get_feature_names():
    print(x)

'
(
)
,
.
1
2
3
;
[
]
gira
ix
kisiel
kvas
kvass
kwas
kysel
kyselica
parovec
xiv
«квасник»
«пир
«праздник
«пьяница»
żur
алкогольный
аловина
белор
брожения
был
в
веками
вощины
времени
готовится
готовят
гуляние»
диал
добавлением
значении
и
из
изготавливали
или
иногда
квас
кислый
который
латыш
лит
лужиц
макед
между
муки
мёда
на
напитком
напиток
нередко
о
он
основе
основным
основой
пахучих
период
пир»
польск
похлёбок
праздниках
раньше
ржаного
с
свадебный
свадьбах
свареный
свидетельствуют
свёклы
сербохорв
славяне
славянский
словацк
слово
служил
солода
ст.-словацк
сухого
также
творёный
того
том
трав
традиционный
угощение
укр
употребляли
фруктов
хлеба
хмельным
холодных
что
ягод
языке
ячменного
—
